In [4]:
import itertools
import time
import matplotlib.pyplot as plt
import numpy as np
import sys
import pandas as pd
from tqdm import tqdm
from random import randint, choice, uniform
from math import ceil
from statistics import stdev, mean
from qiskit import *
from qiskit.visualization import plot_histogram
from IPython.core.display import display, HTML
import json
import numpy as np
class Graph():
    def __init__(self, N, randomize=True):
        ''' Initialize a random graph with N vertices. '''
        self.N = N
        self.E = 0
        self.adj = {n:dict() for n in range(N)}

        # For storing information about each run.
        self.currentScore = float('-inf')
        self.currentBest = ""
        self.runs = []

        # Randomly generate edges
        if randomize:
            self.randomize()

    def randomize(self):
        ''' Randomly generate edges for this graph. '''

        # Generate list of tuples for all possible directed edges.
        all_possible_edges = set([(x,y) for x in range(self.N) for y in range(self.N) if x != y])

        # Sanity check, ensuring we generated the correct number of edges.
        e_gen = len(all_possible_edges) / 2
        e_shd = self.N * (self.N-1) / 2
        assert e_gen == e_shd , "%d != %d" % (e_gen, e_shd)

        # Choose a random number of edges for this graph to have. 
        # Note, we stop at len/2 because we generated directed edges,
        # so each edge counts twice.
        num_edges = len(all_possible_edges)//2
        for i in range(num_edges):
            # Choose an edge, remove it and its directed complement from the list.
            e = choice(list(all_possible_edges))
            all_possible_edges.remove(e)
            all_possible_edges.remove(e[::-1])

            # Unpack tuple into vertex ints.
            u, v = int(e[0]), int(e[1])

            # Choose a random weight for each edge.
            weight = randint(1, 100)

            #weight = 1
            self.add_edge(u, v, weight)


    def add_edge(self, u, v, weight):
        ''' Add an edge to the graph. '''
        self.E += 1
        self.adj[u][v] = weight

    def get_edges(self):
        ''' Get a list of all edges. '''
        edges = []
        for u in self.adj:
            for v in self.adj[u]:
                edges.append((u, v, self.adj[u][v]))
        return edges

    def get_score(self,bitstring):
        ''' Score a candidate solution. '''
        assert len(bitstring) == self.N

        score = 0

        # For every edge u,v in the graph, add the weight
        # of the edge if u,v belong to different cuts
        # given this canddiate solution.

        for u in self.adj:
            for v in self.adj[u]:
                if bitstring[u] != bitstring[v]:
                    score += self.adj[u][v]
        return score

    def optimal_score(self):
        '''
        Returns (score, solutions) holding the best possible solution to the
        MaxCut problem with this graph.
        '''

        best = 0
        best_val = []

        # Iterate over all possible candidate bitstrings
        # Note: the bitstrings from 0 - N/2 are symmetrically
        # equivalent to those above
        for i in range(ceil((2 ** self.N)/2)):
            # Convert number to 0-padded bitstring.
            bitstring = bin(i)[2:]
            bitstring = (self.N - len(bitstring)) * "0" + bitstring

            sc = self.get_score(bitstring)
            if sc > best:
                best = sc
                best_val = [bitstring]
            elif sc == best:
                best_val.append(bitstring)
        return best, best_val

    def edges_cut(self, bitstring):
        ''' Given a candidate solution, return the number of edges that this solution cuts. '''
        num = 0
        for u in self.adj:
            for v in self.adj[u]:
                if bitstring[u] != bitstring[v]:
                    num += 1
        return num

    def update_score(self, bitstring):
        ''' Scores the given bitstring and keeps track of best. '''
        score = self.get_score(bitstring)
        if score > self.currentScore:
            self.currentScore = score
            self.currentBest = bitstring
        return score
    
    def clear_runs(self):
        ''' Clear data from past runs. '''
        self.currentScore = float('-inf')
        self.currentBest = ""
        self.runs = []
        
    def add_run(self, gamma, beta, expected_value):
        ''' Save the data from each run iteration. '''
        self.runs.append([gamma, beta, expected_value])
        
    def __str__(self):
        return "Graph with %d vertices %d edges.\nAdjacency List: %s" % (self.N, self.E, self.adj)
maxB = 0.0
maxG = 0.0
maxExp = 0.0
def get_expectation(x, g, opt, NUM_SHOTS=128):
    # Look for progress bar as a global variable.
    global maxB
    global maxG
    gamma, beta = x
    # Construct quantum circuit.
    q = QuantumRegister(g.N)
    c = ClassicalRegister(g.N)
    qc = QuantumCircuit(q, c)
    # Apply hadamard to all inputs.
    for i in range(g.N):
        qc.h(q[i])
    # Apply V for all edges.
    for edge in g.get_edges():
        u, v, w = edge
        # Apply CNots.
        qc.cx(q[u], q[v])
        qc.u1(gamma*w, q[v])
        # Apply CNots.
        qc.cx(q[u], q[v])
    # Apply W to all vertices.
    for i in range(g.N):
        qc.h(q[i])
        qc.u1(-2*beta, q[i])
        qc.h(q[i])
    # Measure the qubits (avoiding ancilla).
    for i in range(g.N):
        qc.measure(q[i], c[i])
    # Run the simluator.
    backend = BasicAer.get_backend("qasm_simulator")
    job = execute(qc, backend, shots=NUM_SHOTS)
    results = job.result()
    result_dict = results.get_counts(qc)
    # Calculate the expected value of the candidate bitstrings.
    exp = 0
    global maxExp
    for bitstring in result_dict:
        prob = np.float(result_dict[bitstring]) / NUM_SHOTS
        score = g.update_score(bitstring)
        exp += score * prob
        if(exp>maxExp):
            maxB = beta
            maxG = gamma
            maxExp = exp
    print(maxB)
    print(maxG)
    print(maxExp)
    print("-------")
    g.add_run(gamma, beta, exp)
    return exp
ptsB = []
ptsG = []
vals = []
pointsB = []
pointsG = []
def hold_constant():
    ''' Plots expected value vs. gamma/beta, holding the rest of the variables constant.'''
    global ptsB
    global ptsG
    global maxExp
    global pointsB
    global pointsG
    MINB = 0
    MAXB = np.pi
    
    g = Graph(4)
    print(g)
    # RUNS # of runs at each gamma for error bars.
    RUNS = 3

    # Keep track of gammas, expected values, for plotting.
    pts, exp, std = [], [], []

    # The maximum possible expected value is the maximum possible weighted cut.
    opt = g.optimal_score()[0]
    print("Running hold_constant!\n")
    print("Optimal score: %s\n" % (opt))
    
    # Number of data points to collect.
    NUM_RUNS = 100
    ans = list(list())
    MIN = 0
    MAX = 2*np.pi
    pointsB = np.linspace(MINB, MAXB, NUM_RUNS)
    pointsG = np.linspace(MIN, MAX, NUM_RUNS)
    for i in range(100):
        for j in range(100):
            ptsB.append(pointsB[i])
    for i in range(100):
        for j in range(100):
            ptsG.append(pointsG[j])
    for i in itertools.product(pointsB,pointsG):
        pts.append(i)
    for i in range(len(pts)):
        # Calculate expected values.
        params = pts[i]
        exp = vals.append(get_expectation(params,g,opt))
        if(maxExp>=0.9*opt):
            print("Accuracy achieved")
            break
hold_constant()

Graph with 4 vertices 6 edges.
Adjacency List: {0: {}, 1: {0: 83, 2: 95}, 2: {0: 72, 3: 96}, 3: {0: 23, 1: 61}}
Running hold_constant!

Optimal score: 312

0.0
0.0
215.1171875
-------
0.06346651825433926
0.0
221.046875
-------
0.06346651825433926
0.0
221.046875
-------
0.1903995547630178
0.0
227.796875
-------
0.1903995547630178
0.0
227.796875
-------
0.1903995547630178
0.0
227.796875
-------
0.1903995547630178
0.0
227.796875
-------
0.1903995547630178
0.0
227.796875
-------
0.1903995547630178
0.0
227.796875
-------
0.1903995547630178
0.0
227.796875
-------
0.1903995547630178
0.0
227.796875
-------
0.1903995547630178
0.0
227.796875
-------
0.1903995547630178
0.0
227.796875
-------
0.1903995547630178
0.0
227.796875
-------
0.1903995547630178
0.0
227.796875
-------
0.1903995547630178
0.0
227.796875
-------
0.1903995547630178
0.0
227.796875
-------
0.1903995547630178
0.0
227.796875
-------
1.1423973285781066
0.0
228.578125
-------
1.1423973285781066
0.0
228.578125
-------
1.14239732857810

1.1423973285781066
0.0
228.578125
-------
1.1423973285781066
0.0
228.578125
-------
1.1423973285781066
0.0
228.578125
-------
1.1423973285781066
0.0
228.578125
-------
1.1423973285781066
0.0
228.578125
-------
1.1423973285781066
0.0
228.578125
-------
1.1423973285781066
0.0
228.578125
-------
1.1423973285781066
0.0
228.578125
-------
1.1423973285781066
0.0
228.578125
-------
1.1423973285781066
0.0
228.578125
-------
1.1423973285781066
0.0
228.578125
-------
1.1423973285781066
0.0
228.578125
-------
1.1423973285781066
0.0
228.578125
-------
1.1423973285781066
0.0
228.578125
-------
0.3807991095260356
0.06346651825433926
229.75
-------
0.3807991095260356
0.06346651825433926
229.75
-------
0.3807991095260356
0.06346651825433926
229.75
-------
0.5711986642890533
0.06346651825433926
238.3046875
-------
0.5711986642890533
0.06346651825433926
238.3046875
-------
0.6981317007977318
0.06346651825433926
246.5703125
-------
0.6981317007977318
0.06346651825433926
246.5703125
-------
0.825064737306

1.1423973285781066
0.0951997773815089
267.0546875
-------
2.729060284936588
0.0951997773815089
268.5
-------
2.729060284936588
0.0951997773815089
268.5
-------
2.729060284936588
0.0951997773815089
268.5
-------
2.729060284936588
0.0951997773815089
268.5
-------
2.729060284936588
0.0951997773815089
268.5
-------
2.729060284936588
0.0951997773815089
268.5
-------
2.729060284936588
0.0951997773815089
268.5
-------
2.729060284936588
0.0951997773815089
268.5
-------
2.729060284936588
0.0951997773815089
268.5
-------
2.729060284936588
0.0951997773815089
268.5
-------
2.729060284936588
0.0951997773815089
268.5
-------
2.729060284936588
0.0951997773815089
268.5
-------
2.729060284936588
0.0951997773815089
268.5
-------
2.729060284936588
0.0951997773815089
268.5
-------
2.729060284936588
0.0951997773815089
268.5
-------
2.729060284936588
0.0951997773815089
268.5
-------
2.729060284936588
0.0951997773815089
268.5
-------
2.729060284936588
0.0951997773815089
268.5
-------
2.729060284936588
0.0951

5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375


5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375
-------
5.648520124636194
0.12693303650867852
269.84375


...

3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754

3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754

3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754

3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754

3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754105527
280.296875
-------
3.5541250222429985
2.3482611754